In [1]:
import pandas as pd
import pystac
import pystac_client
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc
import plotly.express as px
pc.settings.set_subscription_key('6d4762f1152d42a285532dd26ea62836')
from datetime import datetime, timedelta

In [2]:
train_path = '../data/train.csv'
test_path = '../data/test.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [3]:
train_df

,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Rice Yield (kg/ha)
0,Chau_Phu,10.510542,105.248554,SA,T,15-07-2022,3.40,5500
1,Chau_Phu,10.509150,105.265098,SA,T,15-07-2022,2.43,6000
2,Chau_Phu,10.467721,105.192464,SA,D,15-07-2022,1.95,6400
3,Chau_Phu,10.494453,105.241281,SA,T,15-07-2022,4.30,6000
4,Chau_Phu,10.535058,105.252744,SA,D,14-07-2022,3.30,6400
...,...,...,...,...,...,...,...,...
552,Thoai_Son,10.364419,105.164984,WS,T,12-04-2022,7.80,6640
553,Thoai_Son,10.358094,105.189541,WS,T,12-04-2022,2.00,7200
554,Thoai_Son,10.368014,105.238516,WS,T,12-04-2022,6.20,7200
555,Thoai_Son,10.275419,105.234563,WS,T,20-04-2022,3.00,6400


In [34]:
def get_data(row, history=120, resolution=10, surrounding_box=0.1, num_images=20, image_size=1024): # history = num of days to take before haversting
    longitude = row['Longitude']
    latitude = row['Latitude']
    min_longitude = longitude - surrounding_box / 2
    min_latitude = latitude - surrounding_box / 2
    max_longitude = longitude + surrounding_box / 2
    max_latitude = latitude + surrounding_box / 2
    bbox = [min_longitude, min_latitude, max_longitude, max_latitude]
    
    havest_date = row['Date of Harvest']
    havest_datetime = datetime.strptime(havest_date, '%d-%m-%Y')
    sowing_datetime = havest_datetime - timedelta(days=history)
    time_period = f'{sowing_datetime.strftime("%Y-%m-%d")}/{havest_datetime.strftime("%Y-%m-%d")}'
    
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = catalog.search(collections=["sentinel-2-l2a"], bbox=bbox, datetime=time_period)
    items = list(search.get_all_items())
    
    scale = resolution / 111320.0
    bands=["red", "green", "blue", "nir", "rededge", "B05", "B06", "B07", "SCL"]
    
    data = stac_load(
        items,
        bands=bands,
        crs="EPSG:4326", # Latitude-Longitude
        resolution=scale, # Degrees
        chunks={"x": 2048, "y": 2048},
        dtype="uint16",
        patch_url=pc.sign,
        bbox=bbox
    )
    
    print(data)
    
    times = data.time[-num_images:].values.tolist()
    
    for band in bands:
        for i, time in enumerate(times):
            timestamp = int(time / 1e9)
            date = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d')
            print(i, timestamp, date)

        print(data['red'][-num_images:][0])

In [35]:
train_df.apply(get_data, axis=1)

<xarray.Dataset>
Dimensions:      (latitude: 1114, longitude: 1114, time: 23)
Coordinates:
  * latitude     (latitude) float64 10.56 10.56 10.56 ... 10.46 10.46 10.46
  * longitude    (longitude) float64 105.2 105.2 105.2 ... 105.3 105.3 105.3
    spatial_ref  int32 4326
  * time         (time) datetime64[ns] 2022-03-21T03:15:39.024000 ... 2022-07...
Data variables:
    red          (time, latitude, longitude) uint16 dask.array<chunksize=(1, 1114, 1114), meta=np.ndarray>
    green        (time, latitude, longitude) uint16 dask.array<chunksize=(1, 1114, 1114), meta=np.ndarray>
    blue         (time, latitude, longitude) uint16 dask.array<chunksize=(1, 1114, 1114), meta=np.ndarray>
    nir          (time, latitude, longitude) uint16 dask.array<chunksize=(1, 1114, 1114), meta=np.ndarray>
    rededge      (time, latitude, longitude) uint16 dask.array<chunksize=(1, 1114, 1114), meta=np.ndarray>
    B05          (time, latitude, longitude) uint16 dask.array<chunksize=(1, 1114, 1114), meta=n

<xarray.Dataset>
Dimensions:      (latitude: 1114, longitude: 1114, time: 23)
Coordinates:
  * latitude     (latitude) float64 10.56 10.56 10.56 ... 10.46 10.46 10.46
  * longitude    (longitude) float64 105.2 105.2 105.2 ... 105.3 105.3 105.3
    spatial_ref  int32 4326
  * time         (time) datetime64[ns] 2022-03-21T03:15:39.024000 ... 2022-07...
Data variables:
    red          (time, latitude, longitude) uint16 dask.array<chunksize=(1, 1114, 1114), meta=np.ndarray>
    green        (time, latitude, longitude) uint16 dask.array<chunksize=(1, 1114, 1114), meta=np.ndarray>
    blue         (time, latitude, longitude) uint16 dask.array<chunksize=(1, 1114, 1114), meta=np.ndarray>
    nir          (time, latitude, longitude) uint16 dask.array<chunksize=(1, 1114, 1114), meta=np.ndarray>
    rededge      (time, latitude, longitude) uint16 dask.array<chunksize=(1, 1114, 1114), meta=np.ndarray>
    B05          (time, latitude, longitude) uint16 dask.array<chunksize=(1, 1114, 1114), meta=n

KeyboardInterrupt: 